In [1]:
import pycolmap as pc
from pathlib import Path
import sys
sys.path.append('/src/app')
sys.path.append('/src/')
sys.path.append('src')
from place_utils import localize_places

Using cache found in /root/.cache/torch/hub/yxgeee_OpenIBL_master


In [2]:
feature_conf = "superpoint_aachen"
matcher_conf = "superglue"

In [3]:
import numpy as np
import cv2
import tqdm
import os
from IPython.display import clear_output

# Clear the output of the current cell
clear_output()

def get_qt_vector(recon):
    res = {}
    for k,image in recon.images.items():
        qtvec = np.concatenate((image.qvec,image.tvec), axis=0)
        res[image.name] = qtvec
    return res

def matcher_localize(image_paths, dataset_name, weightNo, num=20, rank=5, cvtColor=None, limit=99999999):
    poses = {}
    count = 0
    for image_path in tqdm.tqdm(image_paths):
        args = {"name": dataset_name,
                "input": image_path,
                "num": num,
                "rank": rank,
                "feature_conf":feature_conf,
                "weight":weightNo,
                "matcher_conf":matcher_conf,}
        print(args)
        image = cv2.imread(image_path)
        if cvtColor: image = cv2.cvtColor(image, cvtColor)
        queries = [(os.path.basename(image_path), image)]
        pose = localize_places(queries, args)
        poses[image_path] = pose
        count+=1
        if count == limit: break
        clear_output()
    return poses

def set_query_poses_to_model(recon, poses):
    """
    use output from matcher localize
    """
    for i, image in recon.images.items():
        image_name = image.name
        q = poses[image_name][0]['orientation']
        t = poses[image_name][0]['position']
        old_qvec = recon.images[i].qvec
        old_tvec = recon.images[i].tvec
        recon.images[i].tvec = np.array([t['x'], t['y'], t['z']])
        recon.images[i].qvec = np.array([q['w'], q['x'], q['y'], q['z']])
    print("set new camera poses success!")

In [4]:
!ls /src/matcher_engine/HierarchicalLocalization/PMUC_outputs/isl2-20221101/weight1

features.h5					     localize_dbqr0
features_matches-superglue_pairs-query-netvlad20.h5  map0
glob-des-netvlad.h5				     matches.h5
hloc_superpoint+superglue_netvlad20.txt		     pairs-query-netvlad20.txt
hloc_superpoint+superglue_netvlad20.txt_logs.pkl


In [4]:
from pathlib import Path
import os

def run_eval_pipeline(map_name, weightNo):
    txt=''
    map_path = Path("/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/") / map_name
    for i in os.scandir(map_path):
        print(i.name)
        if "db" == i.name or "qr" == i.name:
            image_paths = [im.path for im in os.scandir(i)]
            poses = matcher_localize(image_paths, map_name, weightNo, limit=9999999)
            for k,v in poses.items():
                name = '/'.join(k.split("/")[-2:])
                q = v[0]['orientation']
                t = v[0]['position']
                txt += f"{name},{t['x']},{t['y']},{t['z']},{q['w']},{q['x']},{q['y']},{q['z']}\n"
            # print(poses)
            # print([os.path.basename(i) for i in image_paths])
    return txt

# map = "dragon-20221115"
# w = 1
# txt = run_eval_pipeline(map, w)
# open(Path(f"/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/{map}/weight{w}") / "localize_dbqr0", "w").write(txt)

In [6]:
tasks = ["dragon-20221115", "canteen-20221115", "cuparking-20221101"]
weights = [[0],[0],[0]]

for task,wei in zip(tasks, weights):
    map = task
    for we in wei:
        w = we
        txt = run_eval_pipeline(map, w)
        open(Path(f"/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/{map}/weight{w}") / "localize_dbqr0", "w").write(txt)

  0%|          | 1/296 [00:03<19:12,  3.91s/it][2023/02/27 09:25:46.597Z hloc INFO] Select Model Dataset
[2023/02/27 09:25:46.614Z hloc INFO] Extract Local Features
[2023/02/27 09:25:46.615Z hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'features',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2023/02/27 09:25:46.616Z hloc INFO] Start delete query feature: /src/matcher_engine/HierarchicalLocalization/PMUC_outputs/dragon-20221115/weight0/features.h5
[2023/02/27 09:25:46.619Z hloc INFO] Start skip pairs: /src/matcher_engine/HierarchicalLocalization/PMUC_outputs/dragon-20221115/weight0/features.h5
[2023/02/27 09:25:46.620Z hloc INFO] Finished setup model for feature extraction


{'name': 'dragon-20221115', 'input': '/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/dragon-20221115/db/p006-d6-f00163.jpg', 'num': 20, 'rank': 5, 'feature_conf': 'superpoint_aachen', 'weight': 0, 'matcher_conf': 'superglue'}
directory :  /src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/dragon-20221115/images /src/matcher_engine/HierarchicalLocalization/PMUC_outputs/dragon-20221115/weight0 /src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/dragon-20221115



100%|██████████| 1/1 [00:00<00:00, 29.81it/s]
[2023/02/27 09:25:46.659Z hloc INFO] Finished exporting features.
[2023/02/27 09:25:46.660Z hloc INFO] Generate Localize Pairs

100%|██████████| 1/1 [00:00<00:00, 18.41it/s]
[2023/02/27 09:25:46.723Z hloc INFO] Extracting image pairs from a retrieval database.
[2023/02/27 09:25:47.056Z hloc INFO] Found 20 pairs.
[2023/02/27 09:25:47.089Z hloc INFO] Match Query
[2023/02/27 09:25:47.091Z hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}
  0%|          | 1/296 [00:04<22:05,  4.49s/it]


SystemError: <built-in function visit> returned a result with an error set

In [15]:
txtlist = open("/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/isl2-20221101/textlist.txt","r").read()

txtlist = list(filter(lambda x: len(x) > 1, txtlist.split("\n")))

new_txt_file = ''

for i in txtlist:
    for j in txt.split("\n"):
        if j.split(",")[0] == i:
            new_txt_file += j + "\n"
            break

In [17]:
open("/src/matcher_engine/HierarchicalLocalization/PMUC_outputs/isl2-20221101/localize_map0_dbqr0.txt","w").write(new_txt_file)

23063

In [10]:
# import shutil
# for i in os.scandir('/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/'):
#     if os.path.exists(Path(i.path) / "db"):
#         p = Path(i.path)
#         os.makedirs(p / 'images', exist_ok=True)
#         os.makedirs(p / 'images' / "query", exist_ok=True)
#         shutil.copytree(p / 'db', p / 'images' / 'db')       

FileExistsError: [Errno 17] File exists: '/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/essence-20221103/images/db'

In [ ]:
import shutil

improt 